In [1]:
import torch
import transformers

import optiacts

In [2]:
from transformers import BertConfig, BertModel

def print_stats():
    stats = torch.cuda.memory_stats()
    print(f'allocated: {stats["active_bytes.all.current"] / 2**30:.3}Gb, peak: {stats["active_bytes.all.peak"] / 2**30:.3}Gb')

In [3]:
model = BertModel(BertConfig()).cuda()
def run():
    out = model(torch.randint(0, 1000, [16, model.config.max_position_embeddings], device='cuda'))
    print_stats()

In [4]:
print('Model memory:')
print_stats()

Model memory:
allocated: 0.409Gb, peak: 0.409Gb


In [5]:
print('Standard activations memory:')
run()

Standard activations memory:
allocated: 5.12Gb, peak: 5.14Gb


In [6]:
# Replace all activations
for layer in model.encoder.layer:
    layer.intermediate.intermediate_act_fn = optiacts.GELU()
torch.cuda.reset_peak_memory_stats()

print('Memory with optiacts:')
run()

Memory with optiacts:
allocated: 4.03Gb, peak: 4.07Gb
